In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
#df_original = pd.read_csv("./(V1.6)all_products_data_set.csv")
#df.insert(2, "color_info", "")
#df.insert(2, "pattern_info","")
#df.head()

In [3]:

#f = open("./json_files/color_name_map.json")
#colors = json.load(f)
#len(colors.keys())

In [5]:
'''
Input: colors(str), format {"Colours_1", "Colours_2"}
Function: text processing and get the colour information separated by comma ','
Output: Colours(str), format "colours_1, colours_2"
'''
def tc(colors: str)->str:
    temp = (colors.lower().removeprefix("{").removesuffix("}"))
    res = re.sub('"', "", re.sub("/",",",temp))
    return res
#reset df all "colors" colomn format
'''
Input: panda.DataFrame
Function: By using tc function above to reset all cells in 'colors' colomn
Output: None
'''
def reset_colors_format(df: pd.DataFrame):
    for row in range(df.shape[0]):
        if not isinstance(df.loc[row]["colors"], str):
            df.loc[row,'colors'] = ""
            continue
        e = df.loc[row]["colors"]
        c = tc(e)
        df.loc[row,'colors'] = c
''' 
Input: dataframe (panda.Dataframe)
Function: check color cells inside the dataframe row by row, send requery to the colornames.org to get colour hex and save it as the set()
Output: None 
'''
def requery_color_name(df: pd.DataFrame)->None:
    
    for row in range(df.shape[0]):
        res = set()
        pattern = set()
        
        colors_str = df.loc[row]["colors"]
        if colors_str == "":
            continue
        colors_list = df.loc[row]["colors"].split(",")
        #print(row, colors_list)
        for item in colors_list:
            URL = "https://colornames.org/search/results/?type=exact&query="
            URL += item
            try:
                page = requests.get(URL)
                soup = BeautifulSoup(page.content, "html.parser")
                results = soup.find_all("a", class_="button is-fullwidth freshButton")
                
                #the first result which has the highest vote
                content = results[0].find("span").text.strip()
                res.add(content)
            except:
                pattern.add(item)
        df.loc[row, "color_info"] = ",".join(res)
        df.loc[row, "pattern_info"] = ",".join(pattern)


In [37]:
#df_original.columns.get_loc("color_num")

17

In [6]:
# Send requery to the Website colornames.org, and get the hex code of the colour.
# 
main_colors = ["Black", "Blue", "Brown", "Green", \
    "Grey", "Orange", "Pink", "Purple", \
    "Red", "Tan", "White", "Yellow",]
main_color = {}
count = 1
for c in main_colors:
    URL = "https://colornames.org/search/results/?type=exact&query="
    URL += c
    try:
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find_all("a", class_="button is-fullwidth freshButton")
        
        #the first result which has the highest vote
        content = results[0].find("span").text.strip()
        main_color.update({count:content})
        count += 1
        
    except:
        print(c)
print(main_color)   

{1: 'Black (#000000)', 2: 'Blue (#0000ff)', 3: 'Brown (#964b00)', 4: 'Green (#00ff00)', 5: 'Grey (#808080)', 6: 'Orange (#ff8000)', 7: 'Pink (#ffc0cb)', 8: 'Purple (#8e44c6)', 9: 'Red (#ff0000)', 10: 'Tan (#d2b48c)', 11: 'White (#ffffff)', 12: 'Yellow (#ffff00)'}


In [51]:
#Save the current 
#df_original.to_csv("temporary_results.csv", index=False)

In [95]:
#finish color_info from previous manual label
def remodify_df_original(df_original:pd.DataFrame)->None:
    for row in range(df_original.shape[0]):
        color_num = df_original.loc[row]["color_num"]
        color_info = df_original.loc[row]["color_info"]
        #if we don't have the both information just jump to next one
        if not isinstance(color_info, str) and color_num == "0":
            continue
        if color_num == "0":
            continue
        l = color_num.split(",")
        res = []
        if isinstance(color_info, str):
            res = (color_info).split(",")
        for n in l:
            n = int(n)
            try:
                color = main_color.get(n)
                if color == None:
                    continue
                if not res.__contains__(color):
                    res.append(color)
            except:
                print(n)
        df_original.loc[row, "color_info"] = ",".join(res)
        
    

In [98]:
#df_original.to_csv("(V1.6)all_products_data_set.csv", index=False)

In [104]:
# Random sample examination.
#import random
#example = df_original.loc[random.randint(0, df_original.shape[0])]["color_info"]
#print(example)

Purple (#800080),Black (#000000),Navy (#000080),Pink (#ffc0cb),Tan (#d2b48c),Dark Purple (#401b6a),Dusty Rose (#cfa4ae),Green (#00ff00),Rust (#b7410e)


In [7]:
main_color

{1: 'Black (#000000)',
 2: 'Blue (#0000ff)',
 3: 'Brown (#964b00)',
 4: 'Green (#00ff00)',
 5: 'Grey (#808080)',
 6: 'Orange (#ff8000)',
 7: 'Pink (#ffc0cb)',
 8: 'Purple (#8e44c6)',
 9: 'Red (#ff0000)',
 10: 'Tan (#d2b48c)',
 11: 'White (#ffffff)',
 12: 'Yellow (#ffff00)'}

In [13]:
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
from PIL import ImageColor

class Color:
    def __init__(self, color_string:str):
        assert color_string != None
        color_info = color_string.split(" (")
        assert len(color_info) == 2
        self.name = color_info[0]
        self.hex_code = color_info[1].removesuffix(")")
        
    def get_rgb(self)->tuple:
        assert self.hex_code != None
        return ImageColor.getcolor(self.hex_code, "RGB")
        
    def get_info(self) ->str:
        return self.name + " "+ self.hex_code
    
"""
Input: format("color_name": "color_hex_code"), for example, color_1 = (Color class)Navy (#000080), color_2 = (Color class)Blue (#0000ff)
Function: calculate delta_e_cie2000 color distance
Output: Float, the distance 
"""
color_1 = Color("Target (#50a1cd)")
color_2 = Color("Standard Blue (#0000ff)")
def difference(color_1:Color, color_2:Color)->float:
    
    assert color_1 != None and color_2 != None, "color_1 or color_2 is Nonetype"
    color1_set = color_1.get_rgb()
    color1_rgb = sRGBColor(color1_set[0], color1_set[1], color1_set[2])
    color2_set = color_2.get_rgb()
    color2_rgb = sRGBColor(color2_set[0], color2_set[1], color2_set[2])
    #convert from RGB to lab color space
    color1_lab = convert_color(color1_rgb, LabColor)
    color2_lab = convert_color(color2_rgb, LabColor)
    
    return delta_e_cie2000(color1=color1_lab, color2=color2_lab)

print("Example - difference: the distance between {color1} and {color2} is {distance}".format(color1=color_1.get_info(), color2=color_2.get_info(), distance=difference(color_1=color_1, color_2=color_2)))

"""
Input: (Color class) to_be_determined_color
Function: By Comparing with colors_distance in the predetermined color list, we extract the closest main_color.
Output: The cloest color in the list
"""
def get_closest_color(unknown_color:Color)->int:
    assert unknown_color != None
    #sort dict {color_num: int : color_distacne: float}
    temp_distance_map = {}
    for k, v in main_color.items():
        dist = difference (unknown_color, Color(v))
        temp_distance_map.update({k:dist})
        
    sorted_dict = dict(sorted(temp_distance_map.items(), key=lambda item: item[1]))
    
    return min(sorted_dict, key=sorted_dict.get)

print("Example - get_closet_color: {color_to_be_determined} found closest one in {num}, which is {color}".format(color_to_be_determined= color_1.get_info(), num=get_closest_color(color_1), color=main_color.get(get_closest_color(color_1))))

"""
Input: (str) color_info
Function: standarized from the color_info column to color_num
Output: (str)all relative nums (in the relative order)
"""

def from_color_info_to_color_num(color_info:str)->list:
    assert isinstance(color_info, str) and color_info != ""
    res = list()
    color_info_list = color_info.split(",")
    
    for item in color_info_list:
        curr = Color(item)
        res.append(get_closest_color(curr))
    return res
    #",".join(str(e) for e in res)
#print("Example - from_color_info_to_color_num: {Example}\nReturn: {Result}, which means{Decode}".format(Example = example, Result = from_color_info_to_color_num(example), Decode= [main_color.get(c) for c in from_color_info_to_color_num(example)]))

Example - difference: the distance between Target #50a1cd and Standard Blue #0000ff is 83.28111874269814
Example - get_closet_color: Target #50a1cd found closest one in 5, which is Grey (#808080)


In [178]:
"""
Input: pandas.DataFrame
Function: By using color distance algorithm to standardize the color info, and map the res to our main pre-determined 14 colors.
Output: None, but you can use the df.to_csv("temp.csv", index=False) to check the mapping result correctness
"""
def colour_mapping(df_original: pd.DataFrame):
    for row in range(df_original.shape[0]):
        color_info = df_original.loc[row]["color_info"]
        #if color_info is empty
        if not isinstance(color_info, str):
            continue
        try:
            color_num_str = ",".join(str(n) for n in from_color_info_to_color_num(color_info))
            df_original.loc[row, "color_num"] = color_num_str
        except:
            print("Error: ", row, " color_info ", color_info )


In [183]:
#df_original.to_csv("(V1.6)all_products_data_set.csv", index=False)